<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8cf484029e3cfa255e6c0a43d99d1e498624750e494dc362a9713cb5f00c5b53
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-27 16:21:36
-------------------
qualified stocks: 89
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  2.00 K
Current:  1.45 C
-------------------
Today PnL: 71.91 K (0.5%)
Current PnL: -19.02 L (-12.46%)
CY Booked + Current PnL: -6.92 L (-4.54%)
-------------------
Total profit:  2.58 L
Total loss:  -21.60 L
-------------------
Total Booked + Current PnL: 20.06 L (15.97%)
Total Booked PnL: 39.08 L (31.11%)
Curr Year Booked PnL: 12.10 L (8.34%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 86.92 L (59.88%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.63%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
59    RELIANCE  1533.0    -7.66    73.0       X-LC     3.51  231520.0   
86      VOLTAS  1530.0     2.70    59.0       X-MC     3.11  215490.0   
42         ITC   452.0   -36.23    68.0       X-LC     1.70  205698.0   
79  TTKPRESTIG   770.0    96.80    51.0       M-SC     2.06   84750.0   
80  UJJIVANSFB    60.0   131.04    74.0       H-SC    16.33  145800.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
59      21154.0   7617.0        2.24         10.06   3.29  13.68     37.0   
86      23748.0  14007.0       -0.33         12.39   6.50  19.69     99.0   
42       5560.0  15325.0        0.92          2.78   7.45  10.44      4.0   
79     -16027.0  16119.0        0.32        -15.90  19.02   0.09    245.0   
80       3321.0  16198.0        3.81          2.33  11.11  13.70    163.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
59     2.78        1.60    28.24     XY25      NTT  REFINERIES  
86     1.70        1.48    20.02     XY25      NTT          AC  
42     0.36        1.42     8.50      X40      NTT        FMCG  
79    -0.99        0.58    11.34    OX40N      NTT    DURABLES  
80     0.21        1.00    72.08    OX40N      NTT       BANKS

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
64       SATIN   274.00   -22.47    72.0       H-SC     4.14  240581.0   
12   BANKINDIA   190.00   -16.77    74.0       H-MC    15.16  214874.0   
54       QUESS   986.00   -46.56    46.0       X-SC    38.61   53272.0   
80  UJJIVANSFB    60.00   131.04    74.0       H-SC    16.33  145800.0   
37   INDIAMART  4810.62   -53.33    56.0       H-SC     7.02  130120.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
64     -35703.0  178920.0        6.61        -12.92   74.37   51.83    126.0   
12      35066.0   77355.0        4.34         19.50   36.00   62.52     88.0   
54     -11734.0  162661.0        3.91        -18.05  305.34  232.18    198.0   
80       3321.0   16198.0        3.81          2.33   11.11   13.70    163.0   
37       6784.0  124837.0        3.56          5.50   95.94  106.72    119.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
64    -0.20        1.66    17.20     XY24      NTT  FINANCE  
12     0.45        1.48    60.24       XR      NTT    BANKS  
54    -0.07        0.37     3.91     XY24      NTT     MISC  
80     0.21        1.00    72.08    OX40N      NTT    BANKS  
37     0.05        0.90    30.16       AR      ATH     MISC

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
56    RAJOOENG   143.10   -43.74    41.0       H-SC    18.48   85310.0   
20   COFFEEDAY    80.00   -45.67    49.0       L-SC    27.40   81043.0   
50      MASFIN   398.61   -20.51    45.0       H-SC     9.17   90420.0   
83  VAIBHAVGBL   521.00    52.07    55.0       H-SC     4.56  131722.0   
38  INDIGOPNTS  1408.00    90.85    32.0       M-SC     4.91  125290.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
56     -17190.0   57789.0       -2.42        -16.77   67.74  39.61    114.0   
20     -32506.0   72558.0       -2.27        -28.63   89.53  35.27    268.0   
50      -7560.0   29160.0       -2.21         -7.72   32.25  22.05    152.0   
83     -51053.0  162123.0       -2.17        -27.93  123.08  60.77    125.0   
38     -49269.0   49302.0       -2.15        -28.22   39.35   0.02    221.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
56    -0.30        0.59     4.19       AR      ATH       MISC  
20    -0.45        0.56    98.36       XR      NTT     HOTELS  
50    -0.26        0.62    31.43       XR      ATH    FINANCE  
83    -0.31        0.91    17.98       XR      NTT  JEWELLERY  
38    -1.00        0.86     8.98    OX40N      NTT     PAINTS

In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol    FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  Current P/L  \
62   SAIL  228.0    41.94    47.0       M-MC     9.71  223688.0      -1274.0   
36    IEX  219.0   -30.16    62.0       H-SC    17.20  199669.0       -285.0   
88  WIPRO  420.0   -13.79    48.0       M-LC     5.91  151224.0        279.0   

     FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  RRR Ind  \
62  168929.0        0.34         -0.57  75.52  74.53    192.0    -0.01   
36   97518.0        0.06         -0.14  48.84  48.63    115.0    -0.00   
88  109169.0        0.38          0.18  72.19  72.51     53.0     0.00   

    CurrAlloc%  Gained% Criteria Strategy Category  
62        1.54    31.50     XY24      BTT    STEEL  
36        1.38    11.39       XR      NTT     MISC  
88        1.04     6.22       XR      NTT       IT

In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
80  UJJIVANSFB    60.0   131.04    74.0       H-SC    16.33  145800.0   
46   KANSAINER   340.0   -66.87    58.0       H-SC     2.55  227862.0   
79  TTKPRESTIG   770.0    96.80    51.0       M-SC     2.06   84750.0   
74   TATAELXSI  9161.0   -21.53    58.0       H-MC     7.31  106124.0   
68         SIS   528.0  2061.14    50.0       H-SC     3.18   87210.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
80       3321.0  16198.0        3.81          2.33  11.11  13.70    163.0   
46     -41805.0  78134.0       -0.31        -15.50  34.29  13.47    138.0   
79     -16027.0  16119.0        0.32        -15.90  19.02   0.09    245.0   
74     -25711.0  67930.0        0.75        -19.50  64.01  32.03     98.0   
68     -23822.0  47434.0       -0.41        -21.46  54.39  21.26    156.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
80     0.21        1.00    72.08    OX40N      NTT     BANKS  
46    -0.54        1.57    15.86     XY24      NTT    PAINTS  
79    -0.99        0.58    11.34    OX40N      NTT  DURABLES  
74    -0.38        0.73    18.87    OX40N      NTT        IT  
68    -0.50        0.60    17.83    OX40N      NTT      MISC

In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Empty DataFrame
Columns: [Symbol, FTT, Dev%_PE, RSI_14, Conviction, Spread%, Current, Current P/L, FTT Amt, Today P/L%, Current P/L%, FTT%, OTT%, CumlRnk, RRR Ind, CurrAlloc%, Gained%, Criteria, Strategy, Category]
Index: []

In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
80  UJJIVANSFB   60.00   131.04    74.0       H-SC    16.33  145800.0   
57      RECLTD  446.00    45.36    54.0       M-LC     5.68  205535.0   
32    HINDZINC  730.22    28.52    50.0       M-LC    11.18  209800.0   
47    KPIGREEN  731.05    10.56    72.0       H-SC     5.27  131594.0   
81   UNIONBANK  163.00    -8.53    63.0       M-LC     9.34  164820.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
80       3321.0   16198.0        3.81          2.33  11.11  13.70    163.0   
57       2365.0   39771.0        1.55          1.16  19.35  20.74     55.0   
32       4724.0  107837.0       -0.90          2.30  51.40  54.89     52.0   
47       6297.0   52624.0        0.87          5.03  39.99  47.03    141.0   
81      23980.0   20504.0        2.25         17.03  12.44  31.59     66.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
80     0.21        1.00    72.08    OX40N      NTT    BANKS  
57     0.06        1.42     8.11     XY25      NTT  FINANCE  
32     0.04        1.45    26.95      X5K      ATH   METALS  
47     0.12        0.91    64.77       MH      ATH    POWER  
81     1.17        1.14    48.02     XY24      NTT    BANKS

In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
12   BANKINDIA   190.00   -16.77    74.0       H-MC    15.16  214874.0   
80  UJJIVANSFB    60.00   131.04    74.0       H-SC    16.33  145800.0   
87   WHIRLPOOL  2270.00   -38.27    65.0       M-SC     3.55  127990.0   
47    KPIGREEN   731.05    10.56    72.0       H-SC     5.27  131594.0   
37   INDIAMART  4810.62   -53.33    56.0       H-SC     7.02  130120.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
12      35066.0   77355.0        4.34         19.50  36.00   62.52     88.0   
80       3321.0   16198.0        3.81          2.33  11.11   13.70    163.0   
87      13192.0   80851.0        1.50         11.49  63.17   81.92    223.0   
47       6297.0   52624.0        0.87          5.03  39.99   47.03    141.0   
37       6784.0  124837.0        3.56          5.50  95.94  106.72    119.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
12     0.45        1.48    60.24       XR      NTT     BANKS  
80     0.21        1.00    72.08    OX40N      NTT     BANKS  
87     0.16        0.88    51.52       XR      NTT  DURABLES  
47     0.12        0.91    64.77       MH      ATH     POWER  
37     0.05        0.90    30.16       AR      ATH      MISC

In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
75  TATAMOTORS   1065.00   -53.83    16.0       X-LC    15.22  146398.0   
45    JSWINFRA    342.00   -20.48    37.0       X-MC     5.41  199800.0   
24       DMART   5391.45   -16.41    38.0       X-LC    10.11  102190.0   
53        PGHH  17907.65   -30.55    38.0       X-MC     3.03  201105.0   
52     PAGEIND  51769.93   -27.42    40.0       X-MC     6.91   81970.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
75    -130125.0  240195.0       -0.63        -47.06  164.07  39.81     54.0   
45       -547.0   31049.0        0.80         -0.27   15.54  15.23    178.0   
24       -396.0   27203.0        1.04         -0.39   26.62  26.13     38.0   
53        285.0   67511.0       -1.07          0.14   33.57  33.76     80.0   
52      -1540.0   21566.0       -0.10         -1.84   26.31  23.98     82.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
75    -0.54        1.01     3.19     XY24      NTT      AUTO  
45    -0.02        1.38    31.89     X40N      NTT    REALTY  
24    -0.01        0.70    25.09     X40N      ATH      FMCG  
53     0.00        1.39     5.35      X40      ATH      FMCG  
52    -0.07        0.56     3.87      X40      ATH  APPARELS

In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
67  SIEMENS   4671.50    -1.76    51.0       H-LC     0.91  157995.0   
42      ITC    452.00   -36.23    68.0       X-LC     1.70  205698.0   
53     PGHH  17907.65   -30.55    38.0       X-MC     3.03  201105.0   
29  HAVELLS   2069.16    -6.04    45.0       X-MC     3.06  313446.0   
86   VOLTAS   1530.00     2.70    59.0       X-MC     3.11  215490.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
67     -28100.0   75585.0        0.24        -15.10  47.84  25.51     15.0   
42       5560.0   15325.0        0.92          2.78   7.45  10.44      4.0   
53        285.0   67511.0       -1.07          0.14  33.57  33.76     80.0   
29     -14545.0  121084.0       -0.15         -4.43  38.63  32.48     92.0   
86      23748.0   14007.0       -0.33         12.39   6.50  19.69     99.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
67    -0.37        1.09    16.43       AR      ATH  ELECTRICAL  
42     0.36        1.42     8.50      X40      NTT        FMCG  
53     0.00        1.39     5.35      X40      ATH        FMCG  
29    -0.12        2.16     6.48      X40      ATH  ELECTRICAL  
86     1.70        1.48    20.02     XY25      NTT          AC

In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
21      COLPAL   3767.14    -4.98    43.0       X-MC     8.14  221650.0   
75  TATAMOTORS   1065.00   -53.83    16.0       X-LC    15.22  146398.0   
10    BAJAJHFL    181.50   -14.48    45.0       X-MC     5.81  180105.0   
52     PAGEIND  51769.93   -27.42    40.0       X-MC     6.91   81970.0   
54       QUESS    986.00   -46.56    46.0       X-SC    38.61   53272.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
21     -41715.0  155066.0       -1.02        -15.84   69.96   43.04     84.0   
75    -130125.0  240195.0       -0.63        -47.06  164.07   39.81     54.0   
10     -20401.0  117555.0       -0.35        -10.17   65.27   48.45     90.0   
52      -1540.0   21566.0       -0.10         -1.84   26.31   23.98     82.0   
54     -11734.0  162661.0        3.91        -18.05  305.34  232.18    198.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
21    -0.27        1.53     2.90     XY25      ATH      FMCG  
75    -0.54        1.01     3.19     XY24      NTT      AUTO  
10    -0.17        1.24     3.86     X40N      ATH   FINANCE  
52    -0.07        0.56     3.87      X40      ATH  APPARELS  
54    -0.07        0.37     3.91     XY24      NTT      MISC

In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%  Current  \
54       QUESS    986.00   -46.56    46.0       X-SC    38.61  53272.0   
52     PAGEIND  51769.93   -27.42    40.0       X-MC     6.91  81970.0   
58      RELAXO   1176.00   -39.97    42.0       X-SC     4.70  81771.0   
2   ABBOTINDIA  35195.00    -9.64    41.0       X-MC     3.21  89310.0   
13   BATAINDIA   2096.00   -35.29    47.0       X-SC     4.36  96795.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
54     -11734.0  162661.0        3.91        -18.05  305.34  232.18    198.0   
52      -1540.0   21566.0       -0.10         -1.84   26.31   23.98     82.0   
58     -63389.0  139321.0       -2.08        -43.67  170.38   52.31    136.0   
2       -1368.0   16272.0       -0.53         -1.51   18.22   16.44    101.0   
13     -31875.0   77175.0        0.09        -24.77   79.73   35.20    219.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
54    -0.07        0.37     3.91     XY24      NTT      MISC  
52    -0.07        0.56     3.87      X40      ATH  APPARELS  
58    -0.45        0.56     9.45     X40N      NTT  FOOTWEAR  
2     -0.08        0.62    18.36      X40      ATH    PHARMA  
13    -0.41        0.67    11.10      X40      NTT  FOOTWEAR

In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
76    TCS  4389.97   -27.08    59.0       X-LC    12.05  289981.0   
40   INFY  2275.00   -17.80    59.0       X-LC     6.91  320458.0   
42    ITC   452.00   -36.23    68.0       X-LC     1.70  205698.0   
85    VBL   671.64   -18.69    51.0       X-LC     9.80  293065.0   
1     ABB  7934.00   -39.87    53.0       H-LC     6.16  251424.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
76     -55975.0  122691.0        0.71        -16.18  42.31  19.28      1.0   
40       7570.0  164107.0        0.14          2.42  51.21  54.87      3.0   
42       5560.0   15325.0        0.92          2.78   7.45  10.44      4.0   
85     -22777.0  135455.0       -0.44         -7.21  46.22  35.67      5.0   
1      -10195.0  129408.0        1.10         -3.90  51.47  45.57      7.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
76    -0.46        2.00     7.20      X40      ATH          IT  
40     0.05        2.21    10.90      X40      BTT          IT  
42     0.36        1.42     8.50      X40      NTT        FMCG  
85    -0.17        2.02     5.64     X40N      ATH        FMCG  
1     -0.08        1.73     6.98       AR      NTT  ELECTRICAL

In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
7   ASIANTILES   137.00  7072.22    62.0       L-SC     7.31   78695.0   
20   COFFEEDAY    80.00   -45.67    49.0       L-SC    27.40   81043.0   
50      MASFIN   398.61   -20.51    45.0       H-SC     9.17   90420.0   
87   WHIRLPOOL  2270.00   -38.27    65.0       M-SC     3.55  127990.0   
37   INDIAMART  4810.62   -53.33    56.0       H-SC     7.02  130120.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
7      -15115.0   91735.0        0.30        -16.11  116.57   81.67    269.0   
20     -32506.0   72558.0       -2.27        -28.63   89.53   35.27    268.0   
50      -7560.0   29160.0       -2.21         -7.72   32.25   22.05    152.0   
87      13192.0   80851.0        1.50         11.49   63.17   81.92    223.0   
37       6784.0  124837.0        3.56          5.50   95.94  106.72    119.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
7     -0.16        0.54    52.40       XR      NTT  CERAMICS  
20    -0.45        0.56    98.36       XR      NTT    HOTELS  
50    -0.26        0.62    31.43       XR      ATH   FINANCE  
87     0.16        0.88    51.52       XR      NTT  DURABLES  
37     0.05        0.90    30.16       AR      ATH      MISC

In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
63  SAMMAANCAP   326.00  -197.73    81.0       M-SC    26.61  169767.0   
5     ANGELONE  3033.00    19.39    75.0       X-SC     5.57  211314.0   
69    SONACOMS   806.63   -31.93    72.0       M-SC     8.64   84674.0   
60   REPCOHOME   880.00   -54.55    73.0       H-SC     3.49  259189.0   
47    KPIGREEN   731.05    10.56    72.0       H-SC     5.27  131594.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
63      20547.0  123641.0        0.20         13.77   72.83  96.62    208.0   
5       20308.0   37381.0        2.49         10.63   17.69  30.21    157.0   
69     -16485.0   56486.0        1.08        -16.30   66.71  39.54    202.0   
60     -27316.0  284641.0        0.33         -9.53  109.82  89.82    134.0   
47       6297.0   52624.0        0.87          5.03   39.99  47.03    141.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
63     0.17        1.17    83.72     XY25      NTT  FINANCE  
5      0.54        1.46    33.73     X40N      NTT  FINANCE  
69    -0.29        0.58    19.40       AR      ATH     AUTO  
60    -0.10        1.79    37.13     XY24      NTT  FINANCE  
47     0.12        0.91    64.77       MH      ATH    POWER

In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.39
1     25       43.44
2     50       74.22

In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.82
LC    31.99
MC    24.22
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.73
X40      16.17
XY25     12.01
XR       11.26
X40N     10.50
AR        7.75
OX40N     7.43
X200      1.75
X5K       1.45
SR        1.07
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.24
X-LC    21.36
X-MC    18.54
M-SC    12.55
M-LC     5.05
X-SC     4.60
H-LC     4.57
H-MC     3.84
M-MC     1.54
L-SC     1.43
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                14.30         -3.87   37.77
IT                  12.44        -16.51   79.00
FINANCE             12.08         -7.95   56.88
MISC                 6.82        -16.28   80.72
BANKS                6.61         -4.05   58.78
ELECTRICAL           5.75         -8.36   46.78
PAINTS               5.47        -16.29   33.62
INSURANCE            3.78         -0.94   38.57
AUTO                 2.74        -44.19  104.36
FOOTWEAR             2.33        -30.30   82.55
DURABLES             2.27        -16.88   45.31
TRAVEL               2.17        -42.45  113.77
AC                   2.14         -2.51   14.61
CERAMICS             2.12        -30.16   85.33
BREWERIES            1.60         -0.61   21.50
REFINERIES           1.60          9.14    3.29
STEEL                1.54         -0.57   75.52
METALS               1.45          2.25   51.40
ENGINEERING          1.41        -18.56   72.79
REALTY               1.38         -0.27   15.54
CEMENT               1.28        -27.75  109.70
CHEMICALS            1.27        -42.64  159.01
JEWELLERY            1.24        -78.59  140.64
HEALTHCARE           1.15         -8.21   35.38
MINING               1.01         -2.51   25.53
ENTERTAINMENT        0.94        -42.01  126.71
POWER                0.91          4.79   39.99
PHARMA               0.62         -1.53   18.22
APPARELS             0.56         -1.88   26.31
HOTELS               0.56        -40.11   89.53
TEXTILES             0.49        -30.50   67.71

In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3259228.0     22
XR        1314351.0     14
AR        1165396.0      9
X40        812531.0     12
X40N       636941.0      9
OX40N      566347.0     10
XY25       440599.0      8
SR         265643.0      2
X5K        107837.0      1
X200        70277.0      1
MH          52624.0      1

In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        3334843.0     25
M-SC        1219562.0     15
X-MC        1187169.0     14
X-LC        1040015.0     13
X-SC         515477.0      6
H-MC         322918.0      3
M-LC         277281.0      4
H-LC         275270.0      3
L-SC         254130.0      3
M-MC         168929.0      1
L-MC          58707.0      1
L-LC          37473.0      1

In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1144663.0      6
           AR         837148.0      5
           XR         776111.0      7
M-SC       XY24       694024.0      6
X-MC       XY24       580564.0      4
X-LC       X40        446428.0      5
           XY24       310250.0      2
X-MC       X40        288928.0      6
X-SC       X40N       275641.0      4
H-SC       SR         265643.0      2
           OX40N      258654.0      4
X-LC       X40N       212696.0      3
H-LC       AR         204993.0      2
H-MC       XY24       177633.0      1
X-MC       XY25       169073.0      2
M-MC       XY24       168929.0      1
L-SC       XR         164293.0      2
X-SC       XY24       162661.0      1
M-SC       OX40N      149926.0      4
X-MC       X40N       148604.0      2
M-SC       XR         128716.0      2
           XY25       123641.0      1
           AR         123255.0      2
M-LC       XR         109169.0      1
           X5K        107837.0      1
L-SC       OX40N       89837.0      1
H-MC       XR          77355.0      1
X-SC       X40         77175.0      1
X-LC       XY25        70641.0      3
H-LC       X200        70277.0      1
H-MC       OX40N       67930.0      1
L-MC       XR          58707.0      1
H-SC       MH          52624.0      1
M-LC       XY25        39771.0      1
L-LC       XY25        37473.0      1
M-LC       XY24        20504.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
